In [2]:
# 补全路径规划信息 20250128 by chenxinyi

import networkx as nx
import numpy as np
import pandas as pd
import time
import datetime
import copy

import warnings
warnings.filterwarnings("ignore")

In [3]:
def get_path_weight(G, path, weight):
    value = 0
    if len(path) > 1:
        for i in range(len(path)-1):
            u = path[i]
            v = path[i+1]
            value += G.edges[u,v][weight]
    return value


def get_path_stepcoords(path, posname, poscoord):
    points = []
    for p in path:
        index = posname.index(p)
        coord = poscoord[index]
        points.append(coord)
    return points


def get_path_stepbayonet(path, posname, bayonetname):
    bayonet_array = []
    for i in range(0, len(path)):
        try:
            index = posname.index(path[i])
            bayonet = bayonetname[index]
            bayonet_array.append(bayonet)
        except:
            bayonet_array.append(np.nan)
            continue
    return bayonet_array


def get_path_steplength(G, path):
    array = [0]
    if len(path) > 1:
        for i in range(len(path)-1):
            u = path[i]
            v = path[i+1]
            length = round(G.edges[u,v]['length'], 2)
            array.append(length)
    return array

In [4]:
# 读取路网拓扑结构
G = nx.read_gml(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\车辆轨迹提取\center_roadnet')
pos = np.load(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\车辆轨迹提取\center_pos.npy', allow_pickle=True).item()
pos_name = list(pos.keys())
pos_coord = list(pos.values())
realpos = np.load(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\车辆轨迹提取\center_realpos.npy')
realpos_name = list(realpos[0])
bayonet_name = list(realpos[1])

In [5]:
# # 读取重构路径信息
# 
# file1 = '路径重构结果/全部处理结果'  # 路径规划后的结果
# Complete_path_tf = list(np.load(file1 + '/' + 'Complete_path_tf.npy', allow_pickle=True))
# Complete_path_pos = list(np.load(file1 + '/' + 'Complete_path_pos.npy', allow_pickle=True))
# path_pred_travel = list(np.load(file1 + '/' + 'path_pred_travel.npy'))
# path_pos_number = list(np.load(file1 + '/' + 'path_pos_number.npy'))
# 
# # 筛选路径规划成功的出行路径所在列表位置
# positions = [index for index, value in enumerate(Complete_path_tf) if value == 'get']
# 
# file2 = '../车辆轨迹划分/中心城区单次出行轨迹结果'  # 轨迹划分后的结果
# # 单次出行轨迹经过的对应卡口设备
# path_bayonet_file = np.load(file2 + '/' + 'path_bayonet.npy', allow_pickle=True)
# path_bayonet = path_bayonet_file.tolist()
# # 经过每一卡口设备的时间
# path_time_file = np.load(file2 + '/' + 'path_time.npy', allow_pickle=True)
# path_time = path_time_file.tolist()
# # 起始时间
# path_start_time = list(np.load(file2 + '/' + 'path_start_time.npy', allow_pickle=True))
# # 结束时间
# path_stop_time = list(np.load(file2 + '/' + 'path_stop_time.npy', allow_pickle=True))
# # 行程时间
# path_travel = list(np.load(file2 + '/' + 'path_travel.npy'))
# # 车辆号牌
# path_cltmbh = list(np.load(file2 + '/' + 'path_cltmbh.npy'))
# # 原始数据经过的卡口个数
# path_bayonet_number = list(np.load(file2 + '/' + 'path_bayonet_number.npy'))
# 
# print('单次出行路径:', len(Complete_path_tf))
# print('路径规划后的单次出行路径:', len(positions), '占比: {:.2%}'.format(len(positions) / len(Complete_path_tf)))
# 读取重构路径信息
import numpy as np
import pickle

file1 = r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\车辆轨迹提取'  # 路径规划后的结果

# 使用pickle加载路径规划结果
with open(file1 + '/Complete_path_tf.pkl', 'rb') as f:
    Complete_path_tf = pickle.load(f)
with open(file1 + '/Complete_path_pos.pkl', 'rb') as f:
    Complete_path_pos = pickle.load(f)
with open(file1 + '/path_pred_travel.pkl', 'rb') as f:
    path_pred_travel = pickle.load(f)
with open(file1 + '/path_pos_number.pkl', 'rb') as f:
    path_pos_number = pickle.load(f)

# 筛选路径规划成功的出行路径所在列表位置
positions = [index for index, value in enumerate(Complete_path_tf) if value == 'get']

file2 = r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\车辆轨迹提取'  # 轨迹划分后的结果

# 使用numpy加载轨迹划分结果
# 单次出行轨迹经过的对应卡口设备
path_bayonet = np.load(file2 + '/final_path_bayonet.pkl', allow_pickle=True)
# 经过每一卡口设备的时间
path_time = np.load(file2 + '/final_path_time.pkl', allow_pickle=True)
# 起始时间
path_start_time = np.load(file2 + '/final_path_start_time.pkl', allow_pickle=True)
# 结束时间
path_stop_time = np.load(file2 + '/final_path_stop_time.pkl', allow_pickle=True)
# 行程时间
path_travel = np.load(file2 + '/final_path_travel.pkl', allow_pickle=True)
# 车辆号牌
path_cltmbh = np.load(file2 + '/final_path_cltmbh.pkl', allow_pickle=True)
# 原始数据经过的卡口个数
path_bayonet_number = np.load(file2 + '/path_bayonet_number.npy', allow_pickle=True)

# 确保所有加载的数据转换为列表类型
if isinstance(path_bayonet, np.ndarray):
    path_bayonet = path_bayonet.tolist()
if isinstance(path_time, np.ndarray):
    path_time = path_time.tolist()
if isinstance(path_start_time, np.ndarray):
    path_start_time = path_start_time.tolist()
if isinstance(path_stop_time, np.ndarray):
    path_stop_time = path_stop_time.tolist()
if isinstance(path_travel, np.ndarray):
    path_travel = path_travel.tolist()
if isinstance(path_cltmbh, np.ndarray):
    path_cltmbh = path_cltmbh.tolist()
if isinstance(path_bayonet_number, np.ndarray):
    path_bayonet_number = path_bayonet_number.tolist()

print('单次出行路径总数:', len(Complete_path_tf))
print('路径规划成功的出行路径数:', len(positions), 
      '占比: {:.2%}'.format(len(positions) / len(Complete_path_tf)))

In [6]:
# 补全单次出行的总行驶距离（m）、平均速度（km/h）

path_distance = []    # 存储每一次单次出行的总行驶距离 单位:m
path_velocity = []    # 存储每一次单次出行的平均行驶速度 单位:km/h

for i in range(0, len(Complete_path_tf)):
    if Complete_path_tf[i] == "get":
        try:
            total_distance = get_path_weight(G, Complete_path_pos[i], 'length')
            total_distance = round(total_distance, 2)
            velocity = (total_distance / path_travel[i]) * 3.6
            velocity = round(velocity, 2)
            path_distance.append(total_distance)
            path_velocity.append(velocity)
        except:
            print(i, ': Function(get_path_weight) fail')
            Complete_path_tf[i] = -1
            path_distance.append(np.nan)
            path_velocity.append(np.nan)
            continue
    else:
        path_distance.append(np.nan)
        path_velocity.append(np.nan)

print(f'* path_distance records: {len(path_distance)}')
print(path_distance[195000:195010])
print(f'* path_velocity records: {len(path_velocity)}')
print(path_velocity[195000:195010])

In [7]:
# 补全每一路网节点对应的过车时刻、经纬度、行驶距离、卡口编号（没有则为空）

Complete_path_bayonet = []     # 存储每个路网节点对应的卡口编号
Complete_path_coords = []      # 存储每个路网节点的经纬度坐标
Complete_path_distance = []    # 存储从上一个路网节点到当前节点的行驶距离（m）
Complete_path_time = []        # 存储经过每个路网节点的时刻

for i in range(0, len(Complete_path_tf)):
    
    if Complete_path_tf[i] == "get":
        
        # todo 计算距离、经纬度、对应卡口设备
        distances = get_path_steplength(G, Complete_path_pos[i])
        coords = get_path_stepcoords(Complete_path_pos[i], pos_name, pos_coord)
        bayonets = get_path_stepbayonet(Complete_path_pos[i], realpos_name, bayonet_name)
        
        Complete_path_distance.append(distances)
        Complete_path_coords.append(coords)
        Complete_path_bayonet.append(bayonets)
        
        # todo 计算过车时刻和瞬时速度
        start_time = path_time[i][0]
        end_time = path_time[i][-1]
        times = [start_time]
        
        # 遍历每一路网节点
        single_path_pos = Complete_path_pos[i]
        total_dist = path_distance[i]
        total_travel = path_travel[i]
        current_time = start_time
        for j in range(1, len(single_path_pos)):
            # 计算当前节点的行驶距离与单次出行总距离之比
            rate = Complete_path_distance[i][j] / total_dist
            # 根据距离比，等比计算时间差(s)
            trange = round(total_travel * rate)
            # 计算当前节点的过车时间
            current_time = current_time + datetime.timedelta(seconds=trange)
            times.append(current_time)
        
        times[-1] = end_time
        Complete_path_time.append(times)
        
        # 每次出行的数据维度校验
        # print(f"{i}  {len(Complete_path_distance[i])} {len(Complete_path_coords[i])} {len(Complete_path_bayonet[i])} {len(Complete_path_time[i])}" )
        # print("Complete_path_distance:", Complete_path_distance[i])
        # print("Complete_paths_time:", Complete_path_time[i])
        
    else:
        Complete_path_distance.append(np.nan)
        Complete_path_coords.append(np.nan)
        Complete_path_bayonet.append(np.nan)
        Complete_path_time.append(np.nan)
    
    # 定期打印算法运行时间和保存结果
    if i in list(np.arange(20000, 3009117, 20000)):
        print(f"第{i}次出行 信息补全已完成")
print(1)

In [9]:
# 打印特定索引的数据
print(f'* Complete_path_distance records: {len(Complete_path_distance)}')
print(Complete_path_distance[195003])
print(f'* Complete_path_coords records: {len(Complete_path_coords)}')
print(Complete_path_coords[195003])
print(f'* Complete_path_bayonet records: {len(Complete_path_bayonet)}')
print(Complete_path_bayonet[195003])
print(f'* Complete_path_time records: {len(Complete_path_time)}')
print(Complete_path_time[195003])
import pickle # 确保在文件顶部或此代码块附近导入了 pickle

# 保存全部处理结果 (使用 pickle 格式)
with open(file1 + '/' + 'Complete_path_distance.pkl', 'wb') as f:
    pickle.dump(Complete_path_distance, f)

# # 定义批量大小
# batch_size = 10000
# 
# # 分批保存数据
# def save_in_batches(data, filename, batch_size):
#     num_batches = (len(data) + batch_size - 1) // batch_size
#     for i in range(num_batches):
#         start_idx = i * batch_size
#         end_idx = min((i + 1) * batch_size, len(data))
#         batch_filename = f"{filename}_batch_{i}.pkl"
#         with open(batch_filename, 'wb') as f:
#             pickle.dump(data[start_idx:end_idx], f)
#         print(f'Saved batch {i} to {batch_filename}')
# 
# save_in_batches(Complete_path_distance, file1 + '/' + 'Complete_path_distance', batch_size)
# save_in_batches(Complete_path_coords, file1 + '/' + 'Complete_path_coords', batch_size)
# save_in_batches(Complete_path_bayonet, file1 + '/' + 'Complete_path_bayonet', batch_size)
# save_in_batches(Complete_path_time, file1 + '/' + 'Complete_path_time', batch_size)


In [10]:
with open(file1 + '/' + 'Complete_path_coords.pkl', 'wb') as f:
    pickle.dump(Complete_path_coords, f)

In [11]:
with open(file1 + '/' + 'Complete_path_bayonet.pkl', 'wb') as f:
    pickle.dump(Complete_path_bayonet, f)

In [13]:
import sys
sys.getsizeof(Complete_path_time) / (1024**2)  # 单位 MB


In [ ]:
# with open(file1 + '/' + 'Complete_path_time.pkl', 'wb') as f:
#     pickle.dump(Complete_path_time, f)
import joblib
import os

save_path = os.path.join(file1, 'Complete_path_time.pkl')
joblib.dump(Complete_path_time, save_path, compress=3)


In [8]:
print('数据维度检验:')
print(len(path_cltmbh), len(path_travel), len(path_pred_travel), len(path_start_time), len(path_stop_time), \
      len(path_distance), len(path_velocity), len(path_pos_number), \
      len(Complete_path_tf), len(Complete_path_pos), len(Complete_path_coords), len(Complete_path_bayonet), \
      len(Complete_path_time), len(Complete_path_distance))

# 剔除异常出行数据

In [9]:
# # 读取信息补全后的结果
# # file1 = '路径重构结果/全部处理结果'
# # Complete_path_tf = list(np.load(file1 + '/' + 'Complete_path_tf.npy', allow_pickle=True))
# # Complete_path_pos = list(np.load(file1 + '/' + 'Complete_path_pos.npy', allow_pickle=True))
# Complete_path_coords = list(np.load(file1 + '/' + 'Complete_path_coords.npy', allow_pickle=True))
# Complete_path_time = list(np.load(file1 + '/' + 'Complete_path_time.npy', allow_pickle=True))
# Complete_path_distance = list(np.load(file1 + '/' + 'Complete_path_distance.npy', allow_pickle=True))
# Complete_path_bayonet = list(np.load(file1 + '/' + 'Complete_path_bayonet.npy', allow_pickle=True))
# 
# # 保存路径规划结果（仅包括路径规划成功的数据）
# save_file = '路径重构结果/有效处理结果'
# 
# # 车辆号牌
# path_cltmbh = list(np.load('../车辆轨迹划分/中心城区单次出行轨迹结果/path_cltmbh.npy'))
# 
# # 筛选路径规划成功的出行路径所在列表位置
# positions = [index for index, value in enumerate(Complete_path_tf) if value == 'get']
# print('补全信息后的单次出行路径:', len(positions), '占比: {:.2%}'.format(len(positions) / len(Complete_path_tf)))
# 读取信息补全后的结果


# # 使用pickle加载路径规划结果
# with open(file1 + '/Complete_path_tf.pkl', 'rb') as f:
#     Complete_path_tf = pickle.load(f)
# with open(file1 + '/Complete_path_pos.pkl', 'rb') as f:
#     Complete_path_pos = pickle.load(f)
#     
# 加载其他补全信息（假设这些文件也是以pickle格式保存）
# with open(file1 + '/Complete_path_coords.pkl', 'rb') as f:
#     Complete_path_coords = pickle.load(f)
# with open(file1 + '/Complete_path_time.pkl', 'rb') as f:
#     Complete_path_time = pickle.load(f)
# with open(file1 + '/Complete_path_distance.pkl', 'rb') as f:
#     Complete_path_distance = pickle.load(f)
# with open(file1 + '/Complete_path_bayonet.pkl', 'rb') as f:
#     Complete_path_bayonet = pickle.load(f)

# 保存路径规划结果（仅包括路径规划成功的数据）
save_file = file1

# 加载车辆号牌数据（根据之前代码的保存格式）
with open(file1 + '/final_path_cltmbh.pkl', 'rb') as f:
    path_cltmbh = pickle.load(f)

# 筛选路径规划成功的出行路径所在列表位置
positions = [index for index, value in enumerate(Complete_path_tf) if value == 'get']
print('补全信息后的单次出行路径:', len(positions), '占比: {:.2%}'.format(len(positions) / len(Complete_path_tf)))

In [10]:
path_bayonet_number = np.load(file2 + '/path_bayonet_number.npy', allow_pickle=True)

In [11]:
len(path_bayonet_number)

In [12]:
# 创建单次出行信息表

travel_info = {'CLTMBH': [path_cltmbh[i] for i in positions],  
               'start_time': [path_start_time[i] for i in positions],
               'stop_time': [path_stop_time[i] for i in positions], 
               'travel_time_s': [path_travel[i] for i in positions], 
               'predtravel_time_s': [path_pred_travel[i] for i in positions],
               'distance_km': [round(path_distance[i]/1000, 2) for i in positions],
               'speed_kmh': [path_velocity[i] for i in positions],
               'bayonet_records': [path_bayonet_number[i] for i in positions],
               'node_records': [path_pos_number[i] for i in positions], 
               'start_bayonet': [Complete_path_bayonet[i][0] for i in positions], 
               'stop_bayonet': [Complete_path_bayonet[i][-1] for i in positions], 
               'start_node_lon': [Complete_path_coords[i][0][0] for i in positions], 
               'start_node_lat': [Complete_path_coords[i][0][1] for i in positions],
               'stop_node_lon': [Complete_path_coords[i][-1][0] for i in positions], 
               'stop_node_lat': [Complete_path_coords[i][-1][1] for i in positions],
               'position': positions
               }
travel_info = pd.DataFrame(travel_info)

# # 匹配车辆信息表
vehicle_info = pd.read_csv(r'D:\24216373郑金涛中微观研究\中心城区研究车辆信息表.csv', encoding='gbk')
travel_info = pd.merge(travel_info, vehicle_info[['CLTMBH','veh_type','fuel_type','PFBZ']], on='CLTMBH', how='left')
print('travel_info:', len(travel_info))

In [13]:
# todo 剔除平均速度＞60km/h 或 距离＜0.5km的路径
speed_threshold = 60
distance_threshold = 0.5

# 提取有效出行的索引
valid_positions = travel_info.loc[(travel_info['speed_kmh'] <= speed_threshold) & \
                                  (travel_info['distance_km'] >= distance_threshold)]['position'].tolist()

# 统计异常出行
abnormal_records = len(travel_info) - len(valid_positions)
print(f"平均速度＞{speed_threshold}km/h 或 距离＜{distance_threshold}km的异常出行: {abnormal_records}",
      "占比: {:.2%}".format(abnormal_records / len(travel_info)))

# 提取有效出行数据
valid_travel_info = travel_info[travel_info['position'].isin(valid_positions)]
valid_travel_info.reset_index(drop=True, inplace=True)
print(f"剔除速度和距离异常后的出行次数: {len(valid_travel_info)} / {len(travel_info)}", 
      "占比: {:.2%}".format((len(valid_travel_info)) / len(travel_info)))

In [14]:
# todo 剔除估计行程时间与实际行程时间相差较大的路径

# 提取有效出行的索引
def function(a, b):
    minscaler = round(b*0.4)
    maxscaler = round(b*1.5)
    if a > minscaler and a < maxscaler:
        return '总时间误差满足'
    else:
        return '总时间误差不满足'

travel_info['travel_error'] = travel_info.apply(lambda x : function(x['predtravel_time_s'],x['travel_time_s']), axis = 1)
valid_positions2 = travel_info.loc[(travel_info['speed_kmh'] <= speed_threshold) & \
                                   (travel_info['distance_km'] >= distance_threshold) & \
                                   (travel_info['travel_error'] == '总时间误差满足')]['position'].tolist()

abnormal_records2 = len(travel_info) - len(valid_positions2)
print(f"预估行程时间超过 实际行程时间*(0.4~1.5) 区间内的异常出行: {abnormal_records2}",
      "占比: {:.2%}".format(abnormal_records2 / len(travel_info)))

# 提取有效出行数据
valid_travel_info2 = travel_info[travel_info['position'].isin(valid_positions2)]
valid_travel_info2.reset_index(drop=True, inplace=True)

print(f"剔除总行程时间异常后的最终出行次数: {len(valid_travel_info2)} / {len(travel_info)}", 
      "占比: {:.2%}".format((len(valid_travel_info2)) / len(travel_info)))

In [15]:
# todo 统计路网出行信息
print('路网平均运行速度(km/h):', round(np.mean(valid_travel_info2['speed_kmh']), 2), 
      '中位数:', round(np.median(valid_travel_info2['speed_kmh']), 2),
      '最小值:', round(np.min(valid_travel_info2['speed_kmh']), 2),
      '最大值:', round(np.max(valid_travel_info2['speed_kmh']), 2))
print('平均出行距离(km):', round(np.mean(valid_travel_info2['distance_km']), 2),
      '中位数:', round(np.median(valid_travel_info2['distance_km']), 2),
      '最小值:', round(np.min(valid_travel_info2['distance_km']), 2),
      '最大值:', round(np.max(valid_travel_info2['distance_km']), 2))
print('平均出行时间(min):', round(np.mean(valid_travel_info2['travel_time_s']) / 60, 2),
      '中位数:', round(np.median(valid_travel_info2['travel_time_s']) / 60, 2),
      '最小值:', round(np.min(valid_travel_info2['travel_time_s']) / 60, 2),
      '最大值:', round(np.max(valid_travel_info2['travel_time_s']) / 60, 2))

valid_travel_info2.to_csv(save_file + '/' + 'travel_info.csv', encoding='gbk', index=False)
valid_travel_info2.head(3)

In [1]:
# # 单次出行经过的路网节点及其经纬度、过车时刻、对应卡口设备
# 
# nodes_info = [[Complete_path_pos[i] for i in valid_positions2], 
#               [Complete_path_coords[i] for i in valid_positions2], 
#               [Complete_path_time[i] for i in valid_positions2], 
#               [Complete_path_distance[i] for i in valid_positions2], 
#               [Complete_path_bayonet[i] for i in valid_positions2]]
# # # 保存为 .pkl 文件
# # with open(save_file + '/' + "nodes_info.pkl", 'wb') as f:
# #     pickle.dump(nodes_info, f)
# # 定义批次大小
# batch_size = 20000  # 根据你的内存情况调整这个值
# 
# # 获取总长度
# total_length = len(valid_positions2)
# print(f"总数据量: {total_length}")
# 
# # 分批处理并收集所有数据
# all_pos = []
# all_coords = []
# all_time = []
# all_distance = []
# all_bayonet = []
# 
# # 分批处理
# for batch_start in range(0, total_length, batch_size):
#     batch_end = min(batch_start + batch_size, total_length)
#     batch_indices = valid_positions2[batch_start:batch_end]
#     
#     # 处理当前批次
#     batch_pos = [Complete_path_pos[i] for i in batch_indices]
#     batch_coords = [Complete_path_coords[i] for i in batch_indices]
#     batch_time = [Complete_path_time[i] for i in batch_indices]
#     batch_distance = [Complete_path_distance[i] for i in batch_indices]
#     batch_bayonet = [Complete_path_bayonet[i] for i in batch_indices]
#     
#     # 收集批次数据
#     all_pos.extend(batch_pos)
#     all_coords.extend(batch_coords)
#     all_time.extend(batch_time)
#     all_distance.extend(batch_distance)
#     all_bayonet.extend(batch_bayonet)
#     
#     print(f"已处理批次: {batch_start}-{batch_end}")
# 
# # 组装最终数据
# nodes_info = [all_pos, all_coords, all_time, all_distance, all_bayonet]
# 
# # 保存为单个 .pkl 文件
# with open(save_file + '/' + "nodes_info.pkl", 'wb') as f:
#     pickle.dump(nodes_info, f)
# 
# # 打印结果
# print('nodes_info saved to:', save_file + '/' + "nodes_info.pkl")
# print('nodes_info shape:', np.array(nodes_info).shape if len(nodes_info[0]) > 0 else "Empty data")
# 
# # 可选：清理中间变量释放内存
# del all_pos, all_coords, all_time, all_distance, all_bayonet
# 
# # print('* node:', nodes_info[0][0])
# # print('* coord:', nodes_info[1][0])
# # print('* time:', nodes_info[2][0])
# # print('* distance(m):', nodes_info[3][0])
# # print('* bayonet:', nodes_info[4][0])
import pickle
import numpy as np
import os

# 定义批次大小
batch_size = 5000  # 进一步减小批次大小

# 获取总长度
total_length = len(valid_positions2)
print(f"总数据量: {total_length}")

# 使用流式写入方式保存数据
def stream_save_nodes_info(valid_positions2, save_file, batch_size=5000):
    """流式保存节点信息，避免内存溢出"""
    
    os.makedirs(save_file, exist_ok=True)
    
    # 创建临时文件来存储各个部分
    temp_files = []
    part_count = 0
    
    # 分批处理并保存每个部分
    for batch_start in range(0, total_length, batch_size):
        batch_end = min(batch_start + batch_size, total_length)
        batch_indices = valid_positions2[batch_start:batch_end]
        
        # 处理当前批次
        batch_data = [
            [Complete_path_pos[i] for i in batch_indices],
            [Complete_path_coords[i] for i in batch_indices],
            [Complete_path_time[i] for i in batch_indices],
            [Complete_path_distance[i] for i in batch_indices],
            [Complete_path_bayonet[i] for i in batch_indices]
        ]
        
        # 保存临时批次文件
        temp_file = os.path.join(save_file, f"temp_part_{part_count}.pkl")
        with open(temp_file, 'wb') as f:
            pickle.dump(batch_data, f)
        
        temp_files.append(temp_file)
        part_count += 1
        
        print(f"已保存部分: {part_count} (处理了 {batch_end}/{total_length} 条记录)")
    
    print(f"共保存了 {part_count} 个临时部分")
    
    # 逐个读取临时文件并合并保存
    print("开始合并并保存最终文件...")
    
    final_file = os.path.join(save_file, "nodes_info.pkl")
    
    # 分别处理每个数据维度
    with open(final_file, 'wb') as final_f:
        # 创建最终数据结构
        final_data = [[], [], [], [], []]
        
        # 逐个读取临时文件
        for i, temp_file in enumerate(temp_files):
            with open(temp_file, 'rb') as f:
                batch_data = pickle.load(f)
            
            # 扩展最终数据
            for j in range(5):
                final_data[j].extend(batch_data[j])
            
            # 删除临时文件释放空间
            os.remove(temp_file)
            print(f"已处理临时文件 {i+1}/{len(temp_files)}")
        
        # 保存最终数据
        pickle.dump(final_data, final_f)
    
    print('nodes_info saved to:', final_file)
    print('nodes_info shape:', np.array(final_data).shape if len(final_data[0]) > 0 else "Empty data")
    
    return final_file

# 使用流式保存
try:
    saved_file = stream_save_nodes_info(valid_positions2, save_file, batch_size=5000)
    print("保存成功!")
except Exception as e:
    print(f"保存失败: {e}")
    # 如果还是失败，使用纯分批保存方案
    print("使用纯分批保存方案...")
    
    def save_as_separate_batches(valid_positions2, save_file, batch_size=5000):
        """保存为独立的批次文件"""
        
        os.makedirs(save_file, exist_ok=True)
        
        total_length = len(valid_positions2)
        num_batches = (total_length + batch_size - 1) // batch_size
        
        # 保存批次信息
        batch_info = {
            'total_length': total_length,
            'batch_size': batch_size,
            'num_batches': num_batches,
            'data_structure': ['pos', 'coords', 'time', 'distance', 'bayonet']
        }
        
        with open(os.path.join(save_file, "batch_info.pkl"), 'wb') as f:
            pickle.dump(batch_info, f)
        
        # 保存每个批次
        for batch_idx, batch_start in enumerate(range(0, total_length, batch_size)):
            batch_end = min(batch_start + batch_size, total_length)
            batch_indices = valid_positions2[batch_start:batch_end]
            
            batch_data = [
                [Complete_path_pos[i] for i in batch_indices],
                [Complete_path_coords[i] for i in batch_indices],
                [Complete_path_time[i] for i in batch_indices],
                [Complete_path_distance[i] for i in batch_indices],
                [Complete_path_bayonet[i] for i in batch_indices]
            ]
            
            batch_filename = f"nodes_info_batch_{batch_idx:04d}.pkl"
            with open(os.path.join(save_file, batch_filename), 'wb') as f:
                pickle.dump(batch_data, f)
            
            print(f"批次 {batch_idx+1}/{num_batches} 已保存")
        
        print(f"数据已保存为 {num_batches} 个独立批次文件")
        print("使用 load_batched_data(save_file) 来加载数据")
    
    save_as_separate_batches(valid_positions2, save_file, batch_size=5000)
    
    # 提供加载函数
    def load_batched_data(save_path):
        """加载分批保存的数据"""
        # 加载批次信息
        with open(os.path.join(save_path, "batch_info.pkl"), 'rb') as f:
            batch_info = pickle.load(f)
        
        # 合并所有批次
        merged_data = [[] for _ in range(5)]
        
        for batch_idx in range(batch_info['num_batches']):
            batch_filename = f"nodes_info_batch_{batch_idx:04d}.pkl"
            with open(os.path.join(save_path, batch_filename), 'rb') as f:
                batch_data = pickle.load(f)
            
            for i in range(5):
                merged_data[i].extend(batch_data[i])
        
        return merged_data
    
    print("分批保存完成！")
    print("加载数据请使用: nodes_info = load_batched_data(save_file)")

# 创建速度估计的输入数据

In [ ]:
# 新建存储列表
device_id_array = []
datetime_array = []
bayonet_array = []
node_array = []
longitude_array = []
latitude_array = []
travel_id_array = []
c_distance_array = []
distance_array = []

# 遍历每一次补全信息成功的单次出行路径
for i in range(0, len(valid_positions2)):
    
    # 补全信息成功的路径索引
    index = valid_positions2[i]
    travel_id = valid_positions2[i]
    device_id = path_cltmbh[index]
    
    # 遍历经过的每一路网节点（卡口）
    c_distance = 0
    for j in range(0, len(Complete_path_pos[index])):
        datetime = Complete_path_time[index][j]
        node = Complete_path_pos[index][j]
        bayonet = Complete_path_bayonet[index][j]
        longitude = Complete_path_coords[index][j][0]
        latitude = Complete_path_coords[index][j][1]
        distance = Complete_path_distance[index][j]
        c_distance += distance
        # 保存结果
        device_id_array.append(device_id)
        datetime_array.append(datetime)
        bayonet_array.append(bayonet)
        node_array.append(node)
        longitude_array.append(longitude)
        latitude_array.append(latitude)
        travel_id_array.append(travel_id)
        c_distance_array.append(c_distance)
        distance_array.append(distance)
    
    # 定期打印运行结果和保存数据
    if i in list(np.arange(20000, 400001, 20000)):
        print(f"第{i}条路径 记录完成")
        
print(1)

In [ ]:
# 构建dataframe
input_df = {'device_id': device_id_array, 
            'travel_id': travel_id_array, 
            'datetime': datetime_array, 
            'bayonetname': bayonet_array, 
            'posname': node_array, 
            'longitude': longitude_array, 
            'latitude': latitude_array, 
            'c_distance': c_distance_array, 
            'distance': distance_array}
input_df = pd.DataFrame(input_df)

# 保存输入数据
input_df.to_csv(save_file + '/' + 'path_result.csv', index=False, encoding='gbk')

print('速度估计的输入数据量:', len(input_df))
input_df.head(3)

In [ ]:
input_df.info(verbose=False, memory_usage="deep")  # 查看文件大小和占用运行内存

In [ ]:
# 保存输入数据
input_df.to_csv(save_file + '/' + 'path_result.csv', index=False, encoding='gbk')

# 统计路径重构后相邻路网节点之间的时间间隔

In [ ]:
# 计算时间间隔
time_interval = []
for i in range(0, len(Complete_path_time)):
    if Complete_path_tf[i] == "get":
        pathtimes = Complete_path_time[i]
        for j in range(1, len(pathtimes)):
            diff = (pathtimes[j] - pathtimes[j-1]).seconds
            time_interval.append(diff)
    else:
        continue
        
print('最小间隔(s):', min(time_interval))
print('最大间隔(s):', max(time_interval))
print('平均间隔(s):',  round(np.mean(time_interval)))
print('中位数间隔(s):',  round(np.median(time_interval)))

In [ ]:
# 统计时间间隔
interval = [0, 5, 15, 30, 60, 300, 900, 1800, 3600, 28800, 43200, 86400]   # 时间间隔
label = [5, 15, 30, 60, 300, 900, 1800, 3600, 28800, 43200, 86400]
a = pd.cut(time_interval, interval, labels=label)   # 区间左开右闭
counts = pd.value_counts(a)
counts = pd.DataFrame(counts)
counts.reset_index(drop=False, inplace=True)
counts.columns = ['travel_time_s',  'count']
counts.sort_values(by='travel_time_s', ascending=True, inplace=True)  # 按照从小到大排序
counts.reset_index(drop=True, inplace=True)
counts['perc'] = counts['count'] / sum(counts['count'])
counts.to_excel('相邻路网节点的时间间隔分布.xlsx', index=False)  # 保存数据
counts